```{contents}
```
## **External Memory in LangGraph**

In LangGraph, **External Memory** refers to any storage system **outside the in-memory execution state** that preserves information **across runs, sessions, users, and time**.
It enables **long-term learning, personalization, context retention, and system continuity** for production-grade LLM applications.

---

### **1. Why External Memory Is Required**

LLM graphs are inherently **stateless between executions**.
Without external memory, every invocation starts from zero.

External memory enables:

| Capability             | Benefit               |
| ---------------------- | --------------------- |
| Long-term conversation | Persistent assistants |
| Knowledge accumulation | Learning systems      |
| User personalization   | Adaptive behavior     |
| System recovery        | Fault tolerance       |
| Audit & compliance     | Traceability          |

---

### **2. Position in LangGraph Architecture**

```
User Request
   ↓
LangGraph Runtime
   ↓
In-Memory State  ←→  External Memory Store
   ↓
Graph Execution
```

* **State**: Short-term, per-run
* **External Memory**: Long-term, cross-run

---

### **3. Types of External Memory**

| Memory Type        | Purpose          | Technology          |
| ------------------ | ---------------- | ------------------- |
| Checkpoint Store   | Resume execution | Redis, Postgres     |
| Conversation Store | Dialogue history | SQL / NoSQL         |
| Vector Memory      | Semantic recall  | FAISS, Pinecone     |
| Knowledge Store    | Structured facts | SQL                 |
| User Profile Store | Preferences      | Key-value DB        |
| Audit Logs         | Compliance       | Append-only storage |

---

### **4. External Memory vs Internal State**

| Aspect       | State         | External Memory     |
| ------------ | ------------- | ------------------- |
| Lifetime     | One execution | Persistent          |
| Scope        | Thread        | System-wide         |
| Access speed | Very fast     | Slower              |
| Cost         | Free          | Storage cost        |
| Use case     | Control flow  | Knowledge retention |

---

### **5. Workflow: Using External Memory**

#### **Step 1 — Retrieve Memory**

```python
def load_memory(state):
    past = db.get(state["user_id"])
    return {"history": past}
```

#### **Step 2 — Use Memory in Reasoning**

```python
def reason(state):
    context = state["history"] + [state["input"]]
    response = llm(context)
    return {"output": response}
```

#### **Step 3 — Persist Updated Memory**

```python
def save_memory(state):
    db.set(state["user_id"], state["history"] + [state["output"]])
    return {}
```

---

### **6. Vector-Based External Memory**

Used for **semantic recall** instead of raw history.

```python
docs = vector_store.similarity_search(query, k=3)
```

Benefits:

* Retrieves only **relevant** information
* Avoids context window overflow
* Supports knowledge grounding

---

### **7. Production Design Pattern**

```
Request → Load External Memory → Execute Graph → Update Memory → Respond
```

---

### **8. Failure Recovery & Checkpointing**

External memory enables:

* Resume after crash
* Rollback incorrect runs
* Replay execution history

```python
graph.invoke(input, config={"thread_id": "user_123"})
```

---

### **9. Security & Governance**

| Concern            | Control                  |
| ------------------ | ------------------------ |
| User isolation     | Separate memory per user |
| Encryption         | At-rest & in-transit     |
| Retention policies | TTL, pruning             |
| Auditability       | Immutable logs           |
| Access control     | RBAC                     |

---

### **10. When to Use External Memory**

| Scenario           | Required  |
| ------------------ | --------- |
| Chatbots           | Yes       |
| Agents             | Yes       |
| Enterprise systems | Mandatory |
| Short scripts      | No        |

---

### **Mental Model**

> **State is the brain's working memory.
> External memory is long-term memory.**

Together they enable **intelligent, persistent, production-ready LLM systems**.

---

If you want, I can next explain:

• Memory indexing & retrieval strategies
• Memory architectures for multi-agent systems
• Scaling external memory to millions of users

### Demonstration

In [1]:
from typing import TypedDict, List

class State(TypedDict):
    user_id: str
    input: str
    history: List[str]
    output: str

import shelve

memory = shelve.open("memory.db")

def load_memory(user_id):
    return memory.get(user_id, [])

def save_memory(user_id, history):
    memory[user_id] = history


In [2]:
def load_node(state):
    history = load_memory(state["user_id"])
    return {"history": history}

def reason_node(state):
    context = "\n".join(state["history"] + [state["input"]])
    response = f"Assistant remembers: {context}"
    return {"output": response}

def save_node(state):
    new_history = state["history"] + [state["input"], state["output"]]
    save_memory(state["user_id"], new_history)
    return {}



In [3]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("load", load_node)
builder.add_node("reason", reason_node)
builder.add_node("save", save_node)

builder.set_entry_point("load")
builder.add_edge("load", "reason")
builder.add_edge("reason", "save")
builder.add_edge("save", END)

graph = builder.compile()


In [4]:
result = graph.invoke({
    "user_id": "alice",
    "input": "My favorite color is blue."
})
print(result["output"])


Assistant remembers: My favorite color is blue.


In [5]:
result = graph.invoke({
    "user_id": "alice",
    "input": "What is my favorite color?"
})
print(result["output"])


Assistant remembers: My favorite color is blue.
Assistant remembers: My favorite color is blue.
What is my favorite color?
